In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import applications
from keras.layers import Conv2D, MaxPooling2D


In [2]:
img_width, img_height = 224, 224

top_model_weights_path = '128_128_2_2_4classes.h5'#'bottleneck_fc_model_multi_40epoch_6class_3lyr.h5'
train_data_dir = 'pix3d-ML/train'
validation_data_dir = 'pix3d-ML/validation'
test_data_dir = 'pix3d-ML/test'
botfet_train = 'bottleneck_features_train-4class.npy'
botfet_validation = 'bottleneck_features_validation-4class.npy'
num_classes = 4
t = [920,1097,1056,1743]#1049,1245,
z = [192,254,244,350] #sum must be div by 16 #249,244,
nb_train_samples = sum(t)
nb_validation_samples = sum(z)
epochs = 40
batch_size = 16

In [3]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1. / 255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,zoom_range=0.2,fill_mode='nearest')
    datagen_v = ImageDataGenerator(rescale=1. / 255)
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    print('model loaded')
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
#     with open('bottleneck_features_train.npy', 'wb') as f:
#         np.save(f,bottleneck_features_train)
    np.save(open(botfet_train, 'wb'),
            bottleneck_features_train)
    print(bottleneck_features_train.shape)
    generator = datagen_v.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open(botfet_validation, 'wb'),
            bottleneck_features_validation)
#     with open('bottleneck_features_validation.npy', 'wb') as f:
#         np.save(f,bottleneck_features_validation)
    print('done')

In [4]:
save_bottleneck_features() #need to be done once

model loaded
Found 4816 images belonging to 4 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
(4816, 7, 7, 512)
Found 1040 images belonging to 4 classes.
done


In [ ]:
#Take care on unbalanced class
# y_train = np.load(open('label_train.npy','rb'))
y_train = np.array([0]*z[0] + [1]*z[1] + [2]*z[2] + [3]*z[3])
# print(y_train)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

cl = {}
for i in range(len(class_weights)):
    cl[i] = class_weights[i]
print(cl)
# cl={0:1.12869399,1:1.29540481,2:1.0793072,3:0.95100402,4:1.12121212,5:0.67928858}

In [5]:
def train_top_model(cn0,cn1,cn2,dr0,dr1,dr2,outp):
    train_data = np.load(open(botfet_train,'rb'))
#     print(train_data.shape)
    train_labels = []
    tf = []
    for en,item in enumerate(t):
        ii = [0]*num_classes
        ii[en] = 1
#         print(ii,item)
        for tt in range(item):
            train_labels.append(ii)
            tf.append(en)

    train_labels = np.array(train_labels)


    validation_data = np.load(open(botfet_validation,'rb'))
#     validation_labels = np.array([0]*za+[1]*zb+[2]*zc)
    validation_labels = []
    for en,item in enumerate(z):
        ii = [0]*num_classes
        ii[en] = 1
#         print(ii,item)
        for tt in range(item):
            validation_labels.append(ii)
            
    validation_labels = np.array(validation_labels)
#         [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))
    print(validation_data.shape,validation_labels.shape)
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
#     model.add(Dense(cn0, activation='relu'))
#     model.add(Dropout(dr0))
    model.add(Dense(cn1, activation='relu'))
    model.add(Dropout(dr1))
    model.add(Dense(cn2, activation='relu'))
    model.add(Dropout(dr2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              class_weight=cl,
              validation_data=(validation_data, validation_labels))
    model.save_weights(outp)
#     trill(model)
    return model
#     print('done')

In [6]:
train_top_model(0,128,128,0,0.2,0.2,top_model_weights_path)

(1040, 7, 7, 512) (1040, 4)
Epoch 1/40
301/301 [==============================] - 1s 4ms/step - loss: 0.7734 - accuracy: 0.6981 - val_loss: 0.4106 - val_accuracy: 0.8596
Epoch 2/40
301/301 [==============================] - 1s 3ms/step - loss: 0.5012 - accuracy: 0.7973 - val_loss: 0.3048 - val_accuracy: 0.8808
Epoch 3/40
301/301 [==============================] - 1s 3ms/step - loss: 0.4509 - accuracy: 0.8221 - val_loss: 0.2494 - val_accuracy: 0.8962
Epoch 4/40
301/301 [==============================] - 1s 3ms/step - loss: 0.4102 - accuracy: 0.8443 - val_loss: 0.2394 - val_accuracy: 0.9019
Epoch 5/40
301/301 [==============================] - 1s 3ms/step - loss: 0.3335 - accuracy: 0.8623 - val_loss: 0.2501 - val_accuracy: 0.9058
Epoch 6/40
301/301 [==============================] - 1s 3ms/step - loss: 0.2899 - accuracy: 0.8814 - val_loss: 0.1934 - val_accuracy: 0.9183
Epoch 7/40
301/301 [==============================] - 1s 3ms/step - loss: 0.2763 - accuracy: 0.8914 - val_loss: 0.1604 -

#####################################3
[1, 0, 0, 0, 0, 0] 249
[0, 1, 0, 0, 0, 0] 191
[0, 0, 1, 0, 0, 0] 254
[0, 0, 0, 1, 0, 0] 244
[0, 0, 0, 0, 1, 0] 244
[0, 0, 0, 0, 0, 1] 354
(1536, 7, 7, 512) (1536, 6)
Epoch 1/40
444/444 [==============================] - 1s 3ms/step - loss: 1.3516 - accuracy: 0.4331 - val_loss: 0.7567 - val_accuracy: 0.6934
Epoch 2/40
444/444 [==============================] - 1s 3ms/step - loss: 1.0253 - accuracy: 0.5677 - val_loss: 0.5969 - val_accuracy: 0.7617
Epoch 3/40
444/444 [==============================] - 1s 3ms/step - loss: 0.8974 - accuracy: 0.6263 - val_loss: 0.5929 - val_accuracy: 0.7715
Epoch 4/40
444/444 [==============================] - 1s 3ms/step - loss: 0.8512 - accuracy: 0.6437 - val_loss: 0.5120 - val_accuracy: 0.8262
Epoch 5/40
444/444 [==============================] - 2s 4ms/step - loss: 0.8042 - accuracy: 0.6588 - val_loss: 0.6145 - val_accuracy: 0.7507
Epoch 6/40
444/444 [==============================] - 1s 3ms/step - loss: 0.7570 - ac

Epoch 31/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5779 - accuracy: 0.7690 - val_loss: 0.9867 - val_accuracy: 0.6517
Epoch 32/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5343 - accuracy: 0.7855 - val_loss: 0.8191 - val_accuracy: 0.7624
Epoch 33/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5217 - accuracy: 0.7967 - val_loss: 0.8189 - val_accuracy: 0.7533
Epoch 34/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5567 - accuracy: 0.7784 - val_loss: 1.1095 - val_accuracy: 0.6484
Epoch 35/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5426 - accuracy: 0.7790 - val_loss: 1.2690 - val_accuracy: 0.5612
Epoch 36/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5396 - accuracy: 0.7894 - val_loss: 0.8001 - val_accuracy: 0.7741
Epoch 37/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4993 - accuracy: 0.8019 - val_loss: 0.7808 - val_accuracy: 0.7480

Epoch 21/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3496 - accuracy: 0.8757 - val_loss: 0.3724 - val_accuracy: 0.8828
Epoch 22/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3417 - accuracy: 0.8757 - val_loss: 0.4249 - val_accuracy: 0.8724
Epoch 23/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3326 - accuracy: 0.8796 - val_loss: 0.4022 - val_accuracy: 0.8802
Epoch 24/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3154 - accuracy: 0.8863 - val_loss: 0.3344 - val_accuracy: 0.8971
Epoch 25/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3255 - accuracy: 0.8818 - val_loss: 0.4009 - val_accuracy: 0.8757
Epoch 26/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3133 - accuracy: 0.8913 - val_loss: 0.4008 - val_accuracy: 0.8802
Epoch 27/40
444/444 [==============================] - 1s 3ms/step - loss: 0.2724 - accuracy: 0.8995 - val_loss: 0.4647 - val_accuracy: 0.8737

Epoch 37/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3243 - accuracy: 0.8742 - val_loss: 0.3708 - val_accuracy: 0.8861
Epoch 38/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3369 - accuracy: 0.8682 - val_loss: 0.4359 - val_accuracy: 0.8451
Epoch 39/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3357 - accuracy: 0.8656 - val_loss: 0.6412 - val_accuracy: 0.8262
Epoch 40/40
444/444 [==============================] - 1s 3ms/step - loss: 0.2935 - accuracy: 0.8833 - val_loss: 0.4163 - val_accuracy: 0.8607
64 128 512 0.2 0.2 0.2
#####################################3
[1, 0, 0, 0, 0, 0] 249
[0, 1, 0, 0, 0, 0] 191
[0, 0, 1, 0, 0, 0] 254
[0, 0, 0, 1, 0, 0] 244
[0, 0, 0, 0, 1, 0] 244
[0, 0, 0, 0, 0, 1] 354
(1536, 7, 7, 512) (1536, 6)
Epoch 1/40
444/444 [==============================] - 1s 3ms/step - loss: 1.4531 - accuracy: 0.3729 - val_loss: 1.1460 - val_accuracy: 0.4603
Epoch 2/40
444/444 [==============================] - 1s 3

Epoch 27/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4233 - accuracy: 0.8423 - val_loss: 0.4157 - val_accuracy: 0.8424
Epoch 28/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3986 - accuracy: 0.8474 - val_loss: 0.4905 - val_accuracy: 0.8424
Epoch 29/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3958 - accuracy: 0.8529 - val_loss: 0.5063 - val_accuracy: 0.8477
Epoch 30/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4004 - accuracy: 0.8483 - val_loss: 0.5513 - val_accuracy: 0.8333
Epoch 31/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4224 - accuracy: 0.8380 - val_loss: 0.4780 - val_accuracy: 0.8301
Epoch 32/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3919 - accuracy: 0.8515 - val_loss: 0.5542 - val_accuracy: 0.8249
Epoch 33/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3943 - accuracy: 0.8537 - val_loss: 0.4637 - val_accuracy: 0.8496

...

Epoch 27/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4578 - accuracy: 0.8264 - val_loss: 0.4204 - val_accuracy: 0.8711
Epoch 28/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4410 - accuracy: 0.8346 - val_loss: 0.4231 - val_accuracy: 0.8561
Epoch 29/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4518 - accuracy: 0.8319 - val_loss: 0.3720 - val_accuracy: 0.8711
Epoch 30/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4406 - accuracy: 0.8385 - val_loss: 0.3917 - val_accuracy: 0.8633
Epoch 31/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4650 - accuracy: 0.8240 - val_loss: 0.3850 - val_accuracy: 0.8691
Epoch 32/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4354 - accuracy: 0.8366 - val_loss: 0.5752 - val_accuracy: 0.8118
Epoch 33/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4407 - accuracy: 0.8381 - val_loss: 0.3738 - val_accuracy: 0.8665

Epoch 17/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4551 - accuracy: 0.8356 - val_loss: 0.3916 - val_accuracy: 0.8789
Epoch 18/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4246 - accuracy: 0.8487 - val_loss: 0.3476 - val_accuracy: 0.8770
Epoch 19/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4259 - accuracy: 0.8407 - val_loss: 0.4272 - val_accuracy: 0.8796
Epoch 20/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3904 - accuracy: 0.8563 - val_loss: 0.2656 - val_accuracy: 0.9062
Epoch 21/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4067 - accuracy: 0.8546 - val_loss: 0.2725 - val_accuracy: 0.9036
Epoch 22/40
444/444 [==============================] - 1s 3ms/step - loss: 0.4281 - accuracy: 0.8416 - val_loss: 0.3417 - val_accuracy: 0.8939
Epoch 23/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3705 - accuracy: 0.8629 - val_loss: 0.3084 - val_accuracy: 0.8913

Epoch 33/40
444/444 [==============================] - 1s 3ms/step - loss: 0.2674 - accuracy: 0.9054 - val_loss: 0.4086 - val_accuracy: 0.9010
Epoch 34/40
444/444 [==============================] - 1s 3ms/step - loss: 0.2562 - accuracy: 0.9062 - val_loss: 0.4412 - val_accuracy: 0.8913
Epoch 35/40
444/444 [==============================] - 1s 3ms/step - loss: 0.2755 - accuracy: 0.8965 - val_loss: 0.3710 - val_accuracy: 0.8861
Epoch 36/40
444/444 [==============================] - 2s 3ms/step - loss: 0.2899 - accuracy: 0.8934 - val_loss: 0.3993 - val_accuracy: 0.8880
Epoch 37/40
444/444 [==============================] - 1s 3ms/step - loss: 0.3009 - accuracy: 0.8944 - val_loss: 0.4147 - val_accuracy: 0.8848
Epoch 38/40
444/444 [==============================] - 1s 3ms/step - loss: 0.2468 - accuracy: 0.9046 - val_loss: 0.3613 - val_accuracy: 0.8991
Epoch 39/40
444/444 [==============================] - 1s 3ms/step - loss: 0.2394 - accuracy: 0.9120 - val_loss: 0.3737 - val_accuracy: 0.8770

Epoch 23/40
444/444 [==============================] - 1s 3ms/step - loss: 0.9057 - accuracy: 0.6060 - val_loss: 0.8552 - val_accuracy: 0.6478
Epoch 24/40
444/444 [==============================] - 1s 3ms/step - loss: 0.9135 - accuracy: 0.6035 - val_loss: 0.8181 - val_accuracy: 0.6712
Epoch 25/40
444/444 [==============================] - 1s 3ms/step - loss: 0.8864 - accuracy: 0.6208 - val_loss: 0.9073 - val_accuracy: 0.6081
Epoch 26/40
444/444 [==============================] - 1s 3ms/step - loss: 1.0785 - accuracy: 0.5128 - val_loss: 1.0626 - val_accuracy: 0.5163
Epoch 27/40
444/444 [==============================] - 1s 3ms/step - loss: 1.1470 - accuracy: 0.4840 - val_loss: 1.1242 - val_accuracy: 0.4922
Epoch 28/40
444/444 [==============================] - 1s 3ms/step - loss: 1.1311 - accuracy: 0.4958 - val_loss: 1.0628 - val_accuracy: 0.5182
Epoch 29/40
444/444 [==============================] - 1s 3ms/step - loss: 1.1143 - accuracy: 0.4958 - val_loss: 1.1266 - val_accuracy: 0.4967

Epoch 39/40
444/444 [==============================] - 1s 3ms/step - loss: 0.6064 - accuracy: 0.7763 - val_loss: 0.9599 - val_accuracy: 0.6758
Epoch 40/40
444/444 [==============================] - 1s 3ms/step - loss: 0.6269 - accuracy: 0.7651 - val_loss: 0.8267 - val_accuracy: 0.7409
64 64 512 0.3 0.3 0.3
#####################################3
[1, 0, 0, 0, 0, 0] 249
[0, 1, 0, 0, 0, 0] 191
[0, 0, 1, 0, 0, 0] 254
[0, 0, 0, 1, 0, 0] 244
[0, 0, 0, 0, 1, 0] 244
[0, 0, 0, 0, 0, 1] 354
(1536, 7, 7, 512) (1536, 6)
Epoch 1/40
444/444 [==============================] - 1s 3ms/step - loss: 1.5646 - accuracy: 0.3228 - val_loss: 1.1466 - val_accuracy: 0.6348
Epoch 2/40
444/444 [==============================] - 1s 3ms/step - loss: 1.3232 - accuracy: 0.4383 - val_loss: 0.9297 - val_accuracy: 0.7038
Epoch 3/40
444/444 [==============================] - 1s 3ms/step - loss: 1.2534 - accuracy: 0.4724 - val_loss: 0.8664 - val_accuracy: 0.7214
Epoch 4/40
444/444 [==============================] - 1s 3ms/

In [21]:
model = train_top_model(256,256,0.2,0.3)

(7104, 7, 7, 512)
[1, 0, 0, 0, 0, 0] 249
[0, 1, 0, 0, 0, 0] 191
[0, 0, 1, 0, 0, 0] 254
[0, 0, 0, 1, 0, 0] 244
[0, 0, 0, 0, 1, 0] 244
[0, 0, 0, 0, 0, 1] 354
(1536, 7, 7, 512) (1536, 6)
Epoch 1/40
444/444 [==============================] - 2s 4ms/step - loss: 1.2202 - accuracy: 0.5203 - val_loss: 0.5925 - val_accuracy: 0.7637
Epoch 2/40
444/444 [==============================] - 2s 3ms/step - loss: 0.8462 - accuracy: 0.6633 - val_loss: 0.6623 - val_accuracy: 0.7246
Epoch 3/40
444/444 [==============================] - 2s 3ms/step - loss: 0.7091 - accuracy: 0.7233 - val_loss: 0.3738 - val_accuracy: 0.8574
Epoch 4/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5965 - accuracy: 0.7618 - val_loss: 0.4055 - val_accuracy: 0.8581
Epoch 5/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5509 - accuracy: 0.7855 - val_loss: 0.4183 - val_accuracy: 0.8516
Epoch 6/40
444/444 [==============================] - 1s 3ms/step - loss: 0.5330 - accuracy: 0.7976 - val_